# Clize
## A Python approach to argument parsing
Yann Kaiser - [@YannKsr](https://twitter.com/YannKsr)

## A little bit about me

* Full-time student from France

* Started doing Python code in 2011

* Python 3 and metaprogramming maniac!

2011: fixing Python shell script replacements when arch switch to py3

## What is a command-line interface?

    $ git commit -v ./path/to/afile
* It lets you do things
* It takes arguments that change how things are done

## Hello world!

fragment : output  
before next : dave fancies not being Capitalized

In [16]:
def greet(name=None):
    """Returns a greeting for a given person or the whole world."""
    if name is None:
        return "Hello world!"
    else:
        return "Hello {0}!".format(name.title())

In [17]:
greet()

'Hello world!'

In [18]:
greet("john")

'Hello John!'

In [19]:
def greet(name=None, *, no_capitalize=False):
    """Returns a greeting for a given person or the whole world."""
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

In [20]:
greet("dave", no_capitalize=True)

'Hello dave!'

next: Python 2

In [21]:
from sigtools import modifiers

@modifiers.kwoargs('no_capitalize')
def greet(name=None, no_capitalize=False):
    """Returns a greeting for a given person or the whole world."""
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

but further examples Py3

next: argparse answer

In [22]:
greet("dave", no_capitalize=True)

'Hello dave!'

## This is how you write a command-line interface for it
(according to the docs)

In [23]:
import argparse

def greetcli(args=None):
    parser = argparse.ArgumentParser(prog='greeter')
    parser.add_argument("name", nargs='?')
    parser.add_argument("--no-capitalize", action="store_const", const=True)

    args = parser.parse_args(args=args)

    print(greet(args.name, no_capitalize=args.no_capitalize))

greetcli()

usage: greeter [-h] [--no-capitalize] [name]
greeter: error: unrecognized arguments: -f --profile-dir /home/epsy/.ipython/profile_default


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


    $ python greet.py john 

In [24]:
greetcli(['john'])

Hello John!


    $ python greet.py dave --no-capitalize

In [25]:
greetcli(['dave', '--no-capitalize'])

Hello dave!


What did ``greetcli`` do?

### What did ``greetcli`` do?

Positional argument ➡ Positional argument

Named argument ➡ Named argument

This is redundant!

## There has to be a better way!

* The information is right there on ``greet``!

* Python lets us get that information (``inspect.signature``)

## Meet Clize

In [26]:
from clize import run

run(greet)

/home/epsy/.local/lib/python3.4/site-packages/IPython/kernel/__main__.py: Unknown option '-f'
Usage: /home/epsy/.local/lib/python3.4/site-packages/IPython/kernel/__main__.py [OPTIONS] [name]


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [27]:
from clize import run as orig_run

def rrun(*args, **kwargs):
    orig_run(*args, exit=False, **kwargs)

def run(*args, **kwargs): pass

    $ python greeter.py

In [28]:
rrun(greet, args=['greeter.py'])

Hello world!


    $ python greeter.py john

In [29]:
rrun(greet, args=['greeter.py', 'john'])

Hello John!


    $ python greeter.py dave --no-capitalize

In [30]:
rrun(greet, args=['greeter.py', 'dave', '--no-capitalize'])

Hello dave!


    $ python greeter.py --help

In [31]:
rrun(greet, args=['greeter.py', '--help'])

Usage: greeter.py [OPTIONS] [name]

Returns a greeting for a given person or the whole world.

Arguments:
  name

Options:
  --no-capitalize

Other actions:
  -h, --help        Show the help


In [32]:
def greet(name=None, *, no_capitalize=False):
    """Returns a greeting for a given person or the whole world.
    
    name: The name of the person to greet
    
    no_capitalize: Don't capitalize the given name
    """
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

    $ python greet.py --help

In [33]:
rrun(greet, args=['greet.py', '--help'])

Usage: greet.py [OPTIONS] [name]

Returns a greeting for a given person or the whole world.

Arguments:
  name              The name of the person to greet

Options:
  --no-capitalize   Don't capitalize the given name

Other actions:
  -h, --help        Show the help


## More simple things

### Want an int? ➞ Let the default be an int!

In [34]:
def greet(name, *, repeat=1):
    """Returns a greeting for a given person.
    
    name: The name of the person to greet
    
    repeat: Greet the person this many times
    """
    message = "Hello {}!".format(name.title())
    return '\n'.join(message for _ in range(repeat))

    $ python greet.py john --repeat 3

In [35]:
rrun(greet, args=['greet.py', 'john', '--repeat', '3'])

Hello John!
Hello John!
Hello John!


### Collect all positional ➞ Use ``*args``

In [36]:
def greet(*name):
    """Returns a greeting for a given person.
    
    name: The name of the person to greet
    
    repeat: Greet the person this many times
    """
    return "Hello {}!".format(' '.join(name).title())

    $ python greet.py john doe

In [37]:
rrun(greet, args=['greet.py', 'john', 'doe'])

Hello John Doe!


### Want an alias for your option? ➞ Use an annotation

In [38]:
def greet(name=None, *, no_capitalize:'C'=False):
    """Returns a greeting for a given person or the whole world.
    
    name: The name of the person to greet
    
    no_capitalize: Don't capitalize the given name
    """
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

    $ python greet.py dave -C

In [39]:
rrun(greet, args=['greet.py', 'dave', '-C'])

Hello dave!


Use ``sigtools.modifiers.annotate`` on Python 2

Also, many other uses of annotations.

## Run multiple functions

In [40]:
def part(name):
    """Wishes someone goodbye.
    
    name: The person to address"""
    return "Goodbye {}".format(name)

In [41]:
run(greet, part)

    $ python greet.py --help

In [42]:
rrun(greet, part, args=['greet.py', '--help'])

Usage: greet.py command [args...]

Commands:
  greet   Returns a greeting for a given person or the whole world.
  part    Wishes someone goodbye.


Can use iterable, dict, OrderedDict, etc

Share features?

## Share features between commands
### What would argparse do?

Subparsers!

Mixes subcommands and feature sharing together

### What would Python do?

Decorators!

Can be applied to any function arbitrarily

## What would Clize do?

In [43]:
from sigtools.wrappers import wrapper_decorator

@wrapper_decorator(0, 'name')
def with_name(wrapped, name=None, *args, no_capitalize:'C'=False, **kwargs):
    """
    name: The name of the person to address
    
    no_capitalize: Don't capitalize the given name
    """
    if name is None:
        name = 'world'
    elif not no_capitalize:
        name = name.title()
    return wrapped(*args, name=name, **kwargs)

* ``with_name`` becomes a decorator
* ``wrapper_decorator`` makes it so that Clize can know ``*args`` and ``**kwargs`` refer to the decorated function

In [44]:
@with_name
def greet(*, name, repeat:'r'=1):
    """Greets a person or the world.
    
    repeat: How many times should the person be greeted"""
    message = 'Hello {}!'.format(name)
    return '\n'.join(message for _ in range(repeat))

@with_name
def part(*, name):
    """Parts with a person or the world."""
    return 'Goodbye {}.'.format(name)

    $ python greet.py greet -C -r 3 dave

In [45]:
rrun(greet, part, args=['greet.py', 'greet', '-C', '-r', '3', 'dave'])

Hello dave!
Hello dave!
Hello dave!


    $ python greet.py --help

In [46]:
rrun(greet, part, args=['greet.py', 'greet', '--help'])

Usage: greet.py greet [OPTIONS] [name]

Greets a person or the world.

Arguments:
  name               The name of the person to address

Options:
  -r, --repeat=INT   How many times should the person be greeted (default: 1)
  -C, --no-capitalize
                     Don't capitalize the given name

Other actions:
  -h, --help         Show the help


* Repeating the same name is weird, let's greet different people using ``*args``!
* But what about ``--no-capitalize``?

## Creating a decorator for a single argument

In [47]:
from clize import parameters

@parameters.argument_decorator
def name_argument(val, *, no_capitalize:'C'=False):
    """
    For each name:
    
    no_capitalize: Don't capitalize this name
    """
    if not no_capitalize:
        return val.title()
    return val

* ``val`` will receive ``"john"``, ``"dave"`` and so forth
* ``name_argument`` can be used as a parameter annotation

In [48]:
def greet(*names: name_argument):
    if not names:
        return 'Hello world!'
    greetings = []
    for name in names:
        greetings .append('Hello {}!'.format(name))
    return '\n'.join(greetings)

run(greet)

    $ python greet.py john -C dave jane

In [49]:
rrun(greet, args=['greet.py', 'john', '-C', 'dave', 'jane'])

Hello John!
Hello dave!
Hello Jane!


    $ python greet.py --help

In [50]:
rrun(greet, args=['greet.py', '--help'])

Usage: greet.py [OPTIONS] [[-C] names...]

Arguments:
  names...

For each name:
  -C, --no-capitalize
               Don't capitalize this name

Other actions:
  -h, --help   Show the help


## By the way... It's extensible

* Parameter behavior can be replaced or customized.
* ``argument_decorator`` in the above example could have been done by you!

# ``pip install --user clize==3.0b1``

https://clize.readthedocs.org/

[Clize community on Google+](https://plus.google.com/communities/101146333300650079362)

### Yann Kaiser

[@YannKsr](https://twitter.com/YannKsr)

[kaiser.yann@gmail.com](mailto:kaiser.yann@gmail.com)

FAQ page  